In [219]:
import sys
import cv2
from os.path import isfile, join
import numpy as np
from typing import Dict, List, Tuple, Callable
from matplotlib import pyplot as plt
from sewar.full_ref import ssim as ssim_sewar
from skimage.metrics import structural_similarity as ssim_skimage
from external.fast_ssim.ssim import SSIM as ssim_external

In [209]:
screen_dir = '../obs/screenshots/'
filename = 'radhuset station le - BARCODE vs zatic 2024-03-23 15-15-41.png'

filepath = join(screen_dir, filename)

cvflags = cv2.IMREAD_GRAYSCALE
cvflags = None

In [210]:
def show(roi):
    cv2.imshow('roi', roi)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [211]:
# player left

def get_left_roi(image):
    x, y, w, h = 221, 560, 105, 105
    return image[y : y + h, x : x + w]



In [212]:
# player right


def get_right_roi(image):
    x, y, w, h = 2235, 560, 105, 105
    return image[y : y + h, x : x + w]


In [213]:
def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])

	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

def ssim(imageA, imageB):
	return ssim_sewar(imageA, imageB)[0]

In [229]:
reference = cv2.imread(join(screen_dir, filename), flags=cvflags)
left = get_left_roi(reference)
right = get_right_roi(reference)
#show(left)
zatic = left

In [230]:
kats = {
    "kat1" : 'solaris le - Zatic vs BARCODE 2024-01-14 20-32-50.png',
    "kat2" : 'site delta le - BARCODE vs zatic 2024-03-03 16-46-46.png',
    "kat_master" : 'radhuset station le - BARCODE vs zatic 2024-01-09 17-03-09.png',
    "kat_unranked" : 'oceanborn le - Zatic vs BARCODE 2024-03-23 13-54-11.png'
}

kat1_barcode = get_right_roi(cv2.imread(join(screen_dir, kats["kat1"]), flags=cvflags) )
zatic_barcode = get_left_roi(cv2.imread(join(screen_dir, kats["kat1"]), flags=cvflags) )

display(mse(zatic, kat1_barcode))
display(ssim(zatic, kat1_barcode))
#display(ssim_skimage(zatic, kat1_barcode))
display(ssim_sewar(zatic, kat1_barcode))
display(ssim_external(zatic, kat1_barcode))
display(ssim_external(zatic, zatic_barcode))

imgs: Dict[str, Tuple] = {}

for kat, filename in kats.items() :
    img = cv2.imread(join(screen_dir, filename), flags=cvflags)
    imgs[kat] = (get_left_roi(img), get_right_roi(img))



4490.519365079365

0.4010339469878972

(0.4010339469878972, 0.4271641741763302)

0.41799232363700867

0.9997924566268921

In [241]:
# compare left and right regions of screenshots

def show_comparison(kat1, img1, kat2, img2, metric: Callable=ssim_external, threshold=0.95, noshow=True):
    
    
    if metric(img1, zatic) > threshold or metric(img2, zatic) > threshold:
        return
    error = metric(img1, img2)
    if int(error) == 1:
        return
        pass
    
    if noshow:
        display(kat1, kat2, error)
    else:
        plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
        plt.title(kat1)
        plt.show()
        plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
        plt.title(kat2)
        plt.show()
        display(error)


for kat, (left, right) in imgs.items():
    for kat2, (left2, right2) in imgs.items():
        show_comparison(kat, left, kat2, left2)
        show_comparison(kat, right, kat2, right2)
        show_comparison(kat, left, kat2, right2)
        show_comparison(kat, right, kat2, left2)


'kat1'

'kat2'

0.9979430437088013

'kat1'

'kat_master'

...

'kat2'

0.8557567596435547

'kat_unranked'

'kat_master'

0.7184597253799438